In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：用于批量预测的AutoML表格预测模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_tabular_forecasting_batch.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK创建表格预测模型，并使用Google Cloud AutoML 模型进行批量预测。

了解更多关于表格数据预测的信息，请查看[表格数据预测](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/overview)。

### 目标

在本教程中，您将学习如何从Python脚本中创建一个`AutoML`表格预测模型，并使用Vertex AI SDK进行批量预测。您也可以使用`gcloud`命令行工具或通过云控制台在线创建和部署模型。

本教程使用以下Google Cloud ML服务：

- `AutoML训练`
- `Vertex AI批量预测`
- `Vertex AI模型`资源

执行的步骤包括：

- 创建一个`Vertex AI数据集`资源。
- 训练一个`AutoML`表格预测`模型`资源。
- 获取`模型`资源的评估指标。
- 进行批量预测。

数据集

本教程使用的数据集是一个时间序列数据集，其中包含从爱荷华州酒类零售销售数据集中抽取的样本。 数据由爱荷华州商务部提供。 它根据知识共享0 v1.0通用许可证进行提供。 有关更多详细信息，请参见：https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales。 该数据集不需要任何特征工程。 您在本教程中使用的数据集版本存储在BigQuery中。

### 成本

本教程使用了 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您预期的使用情况生成成本估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform

### 仅限Colab：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行认证。请按照下面的相关说明操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经认证通过了。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上给您的服务帐户授予云存储权限。

### 创建一个云存储存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库并定义常量

In [ ]:
import urllib

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

## 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

现在您已经准备好开始创建自己的AutoML表格式预测模型了。

#### BigQuery训练数据的位置。

现在将变量`TRAINING_DATASET_BQ_PATH`设置为BigQuery表的位置。

In [ ]:
TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

创建数据集

接下来，使用`TimeSeriesDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：用于`Dataset`资源的人类可读的名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源。

此操作可能需要几分钟。

In [ ]:
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train",
    bq_source=[TRAINING_DATASET_BQ_PATH],
)

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

print(dataset.resource_name)

In [ ]:
COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

### 创建并运行训练任务

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练任务，2）运行这个任务。

#### 创建训练任务

使用`AutoMLForecastingTrainingJob`类创建一个AutoML训练任务，包括以下参数：

- `display_name`：`TrainingJob`资源的可读性名称。
- `column_transformations`：（可选）要应用于输入列的转换
- `optimization_objective`：要最小化或最大化的优化目标。
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化后的对象是训练流程的任务。

In [ ]:
MODEL_DISPLAY_NAME = "iowa-liquor-sales-forecast-model"

training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

#### 运行训练管道

接下来，您可以通过调用方法`run`来开始训练作业，参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `target_column`：作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `time_column`：用于预测模型的时间序列列。
- `time_series_identifier_column`：时间序列列的ID列。

当完整运行`run`方法后，将返回`Model`资源。

训练管道的执行将需要最多一个小时。

In [ ]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name=MODEL_DISPLAY_NAME,
    predefined_split_column_name=None,
)

## 查看模型评估分数

在模型训练完成后，您可以查看评估分数。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

发送一个批量预测请求

向部署的模型发送一个批量预测。

### 发起批量预测请求

现在您的模型资源已经训练完成，您可以通过调用`batch_predict()`方法使用BigQuery源和目的地来进行批量预测，具有以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `bigquery_source`：BigQuery URI指向一个表，最长为2000个字符。例如：`bq://projectId.bqDatasetId.bqTableId`
- `bigquery_destination_prefix`：用于存储批量预测结果的BigQuery数据集或表。
- `instances_format`：输入实例的格式。由于这里使用了BigQuery源，因此应将其设置为`bigquery`。
- `predictions_format`：输出预测的格式，此处使用`bigquery`以输出到一个BigQuery表。
- `generate_explanations`：设置为`True`以生成解释。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions"
batch_predict_bq_output_dataset_path = "{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client(project=PROJECT_ID)
bq_dataset_id = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_region = "US"  # @param {type : "string"}
bq_dataset_id.location = dataset_region
# delete any existing dataset
try:
    client.delete_dataset(bq_dataset_id, delete_contents=True)
except Exception as e:
    print(e)
bq_dataset = client.create_dataset(bq_dataset_id)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_region
    )
)

对于AutoML模型，可以通过设置最小和最大节点数即`starting_replica_count`和`max_replica_count`为相同的值（在此示例中设置为1）来手动调整扩展。节点数量可以根据负载的需要增加或减少。

`batch_predict`可以将预测结果导出到BigQuery或GCS。此示例将结果导出到BigQuery。

In [ ]:
PREDICTION_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2021_sales_predict"
)

batch_prediction_job = model.batch_predict(
    job_display_name="iowa_liquor_sales_forecasting_predictions",
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=False,
)

print(batch_prediction_job)

### 等待批量预测作业完成

接下来，等待批处理作业完成。或者，您可以在`batch_predict（）`方法中将参数`sync`设置为`True`，以阻塞直到批预测作业完成。

In [ ]:
batch_prediction_job.wait()

获取预测和解释

接下来，获取已完成的批量预测作业的结果并打印出来。每个结果行都将包括预测和解释。

In [ ]:
for row in batch_prediction_job.iter_outputs():
    print(row)

### 可视化预测

最后，请点击以下链接在[数据工作室](https://support.google.com/datastudio/answer/6283323?hl=en)中查看生成的预测。
此部分包含的代码块会动态生成一个数据工作室链接，指定了模板、预测位置和生成图表的查询。数据是从使用BigQuery选项生成的预测中填充的，目的地数据集为`batch_predict_bq_output_dataset_path`。

您可以在https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8检查所使用的模板。这是由Google专门创建的，用来查看预测预测。

**注意：** 只有当使用BigQuery选项成功运行`batch_predict`作业时，数据工作室仪表板才能正确显示图表。

In [ ]:
tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        PREDICTION_DATASET_BQ_PATH,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- AutoML训练作业
- 模型
- 批处理预测作业
- Cloud Storage存储桶

In [ ]:
# Delete dataset
dataset.delete()

# Training job
training_job.delete()

# Delete model
model.delete()

# Delete batch prediction job
batch_prediction_job.delete()

# Delete the dataset
try:
    client.delete_dataset(bq_dataset_id)
except Exception as e:
    print(e)

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI